# 005 Hash functions and pseudo-random number generation

This notebook uses the low-level phylogenetic library [biomcmc-lib](https://github.com/quadram-institute-bioscience/biomcmc-lib) (commit [5975331](https://github.com/quadram-institute-bioscience/biomcmc-lib/commit/5975331ef88d1c4ec9aef9599fb6348905d289c7)).


In [64]:
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/biomcmc-lib/lib
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build.191216/biomcmc-lib/lib
//%cflags: /usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build.191216/biomcmc-lib/lib/.libs/libbiomcmc.a
//%cflags:-lm
#include <biomcmc.h>

int
main (int argc, char **argv)
{
    uint32_t i,j;
    uint32_t seeds[]={0,1,2,3}, vec[32], nvec=34;
    uint8_t *c;
    size_t size = sizeof (vec);
    
    for (i=0;i<2;i++) {
        seeds[0] = i;
        for (j=0;j<4;j++) printf ("%8x ",seeds[j]);
        printf ("original seeds\n");
        biomcmc_salt_vector32_from_spice_table (vec, nvec, seeds);
        printf("\nseed: %u\n", i);
        //for (j=0; j<nvec;j++) {printf("%12u ", vec[j]); if (!((j+1)%8)) printf ("\n"); }
        c = (uint8_t*) vec;
        size = sizeof (vec);
        for (; (size > 0); c++, size--) {printf ("%4x ", *c); if (!((size-1)%16)) printf ("\n"); }
    }
    for (i=0;i<15;i++) {
        for (j=0;j<4;j++) printf ("%8x ",biomcmc_hashint_salted (j, i));
        printf ("<< for hash %u\n", i);
    }
        
    return EXIT_SUCCESS;
}

       0        1        2        3 original seeds
  0:        0  1:        1  2:        2  3:        3  ::DEBUGSEED
  0: c0a9496a  1: b48681b6  2: 25ac1fe5  3:   7f181c  ::DEBUGSEED
  0: b74d7928  1: fa9384d8  2: 58a14bfb  3: b62272f6  ::DEBUGSEED
  0: 2d05ff9e  1: b621b33c  2: 9cc8151d  3: 5ab560c5  ::finalSEED

seed: 0
  15   16   16   b4   31   aa   fc   2e    8   e1   47   99   c5   97   a4   c1 
  1c   92   1d   57   11   cc    f   cf   d7   46   bc   31   87   e1   6f   1f 
  c3   2f   8f   43   99   da   38   db    0   10   46   f7   cf   82   64   17 
  12   85   de   24   12    b   9a   5c   c0    c   63   d5   b0   c7    e   58 
  ae   f5   4c   ae   f7   70   1b   e4   a2   ac   48   c0   f0   79   b0   a7 
  7f   29   eb   ff   ff   bc   cc   ff   8e   dc   13   d8   52   af    2   97 
  4d   a4   58   3c   d8   55   69   1c   b1   71   e3   52   c4   75    e    1 
  85   ff    3   94   91   5f    2   80   e2   ee    5    0   74   80    1    0 
       1 b621b33c 9cc8151d 5

## Printing colours

This week I've been refactoring the random and hash modules of `biomcmc-lib`. I am also adding user-friendly functionality &mdash; linux fortunes and coloured printing. Here we can see my test with colours in `printf()`.

In [26]:
#include <stdio.h>
const char *mytext[] = {"this has ` , $ percent \% and ' while", "so  \%d ____^H^H^H^Hmany"};
const char col_reset[] = "\e[0m";
const char *rownames[7] = {"regular", "bold", "underline", "background", "highbckgrnd", "hightext", "boldhightxt"};
const char *prt_col[][8]={ // 0-black   1-red   2-grn   3-yel   4-blu   5-mag   6-cyn   7-white
   {"\e[0;30m",  "\e[0;31m",  "\e[0;32m",  "\e[0;33m",  "\e[0;34m",  "\e[0;35m",  "\e[0;36m",  "\e[0;37m"},  // 0 regular text
   {"\e[1;30m",  "\e[1;31m",  "\e[1;32m",  "\e[1;33m",  "\e[1;34m",  "\e[1;35m",  "\e[1;36m",  "\e[1;37m"},  // 1 regular bold text
   {"\e[4;30m",  "\e[4;31m",  "\e[4;32m",  "\e[4;33m",  "\e[4;34m",  "\e[4;35m",  "\e[4;36m",  "\e[4;37m"},  // 2 regular underline text
   {"\e[40m",    "\e[41m",    "\e[42m",    "\e[43m",    "\e[44m",    "\e[45m",    "\e[46m",    "\e[47m"},    // 3 regular background
   {"\e[0;100m", "\e[0;101m", "\e[0;102m", "\e[0;103m", "\e[0;104m", "\e[0;105m", "\e[0;106m", "\e[0;107m"}, // 4 high intensity underground
   {"\e[0;90m",  "\e[0;91m",  "\e[0;92m",  "\e[0;93m",  "\e[0;94m",  "\e[0;95m",  "\e[0;96m",  "\e[0;97m"},  // 5 high intensity text 
   {"\e[1;90m",  "\e[1;91m",  "\e[1;92m",  "\e[1;93m",  "\e[1;94m",  "\e[1;95m",  "\e[1;96m",  "\e[1;97m"}   // 6 bold high intensity text
};

int main() {
    int i,j;
    for (i=0;i<8;i++) {
        printf ("\n%d\n", i);
        for (j=0; j < 7; j++) printf ("%15s : %s>%s<>%s<%s\n",rownames[j],prt_col[j][i],mytext[0], mytext[1], col_reset);
    };
    // test with marsaglia constants (should print prime numbers)
    printf ("%5d %5d %5d", (18000<<16)-1, (18000<<15)-1, (18030<<16)-1);
}


0
        regular : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
           bold : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
      underline : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
     background : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
    highbckgrnd : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
       hightext : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
    boldhightxt : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<

1
        regular : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
           bold : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
      underline : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
     background : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
    highbckgrnd : >this has ` , $ percent % and ' while<>so  %d ____^H^H^H^Hmany<
       hig

And this is the same output in my terminal (black background):
<img src="../figures/191213.colourprint.png" align="center" width="500">